In [2]:
import h5py
import os
import glob
import shutil
import matplotlib.pyplot as plt
%matplotlib notebook
import seaborn as sns
import pandas as pd
import statsmodels.stats.multitest as multitest

import pylab as pl
import numpy as np
import scipy.stats as stats
def findOccurrences(s, ch):
    return [i for i, letter in enumerate(s) if letter == ch]

In [2]:
all_session_list = ['20190909_JC110','20190911_JC110','20190913_JC110','20190916_JC110',\
                    '20190918_JC110','20190920_JC110','20190925_JC110','20190930_JC110',\
                    '20191003_JC110','20190927_JC111','20191002_JC111','20191004_JC111',\
                   '20191016_JC113','20191019_JC113','20191021_JC113','20191023_JC113',\
                   '20191025_JC113','20191029_JC113','20191101_JC113','20191106_JC113',\
                   '20191024_JC117','20191028_JC117','20191030_JC117','20191107_JC117',\
                   '20191118_JC117','20191115_JC120','20191116_JC120','20191124_JC120',\
                   '20191125_JC120','20191126_JC120','20191127_JC120','20191119_JC121',\
                   '20190619_JC080','20190624_JC085','20190704_JC085','20190712_JC085',\
                   '20190621_JC091','20190625_JC091','20190701_JC091','20190703_JC091',\
                   '20190705_JC091','20190621_JC097','20190628_JC097','20190702_JC097',\
                   '20190704_JC097','20190708_JC097']

# all_session_list = ['20190909_JC110','20190911_JC110']



#''
class struct: pass
opts = struct()
opts = struct()
opts.rootdir = '/n/coxfs01/2p-data'
opts.acquisition = 'FOV1_zoom4p0x'
opts.traceid = 'traces102'
opts.combined_run = 'scenes_combined'
dst_root = '/n/coxfs01/cechavarria/2p-aggregate/scenes'
traceid = '%s_s2p'%(opts.traceid)

resp_type = 'mean_response'#mean_response or max_response
data_type = 'global_df_f'#'dspks'#'global_df_f'

In [3]:
sess_idx_array = []
all_pval = []
all_tstat_sign = []
ntrials_array = []

# sess_idx = 0
# sess = all_session_list[sess_idx]

for sess_idx, sess in enumerate(all_session_list):

    print(sess)

    i1 = findOccurrences(sess,'_')[0]

    opts.animalid = sess[i1+1:]
    opts.session = sess[0:i1]

    #% Set up source paths:    
    acquisition_dir = os.path.join(opts.rootdir, opts.animalid, opts.session, opts.acquisition)
    responses_dir = os.path.join(acquisition_dir, opts.combined_run,'responses',traceid)
    src_file_dir = os.path.join(responses_dir, 'files')

    #*get trial filter file*
    filt_fn = 'trial_filter.hdf5'
    filt_filepath = os.path.join(src_file_dir, filt_fn)
    filt_grp = h5py.File(filt_filepath, 'r')

    #unpack trial filter
    trial_filter = np.array(filt_grp['trial_filter_matched']).astype('bool')
    trials_to_keep = np.array(filt_grp.attrs['matched_trials'])

    print('**All configs matched to have %i trials**'%(trials_to_keep))
    #unpack trial and config details
    cfg_img = filt_grp['config_img']
    cfg_cond = filt_grp['config_cond']

    filt_grp.close()


    #get response file
    data_array_fn = 'trial_response_array.hdf5'
    resp_filepath = os.path.join(src_file_dir, data_array_fn)

    print('Opening:%s'%(resp_filepath))
    data_grp = h5py.File(resp_filepath, 'r')

    nrois = data_grp.attrs['nrois']
    print('ROIs:%i'%(nrois))

    if 's2p_cell_rois' in data_grp.attrs.keys():
        cell_rois = data_grp.attrs['s2p_cell_rois']
    else:
        cell_rois = np.arange(nrois)
    nrois = len(cell_rois)
    print('Cells:%i'%(nrois))


    curr_slice = 'Slice01'#hard,coding for now

    nconfigs = np.unique(np.array(data_grp['trial_config'])).size
    trial_config = np.array(data_grp['trial_config']).astype('int')



    #consider cell rois only
    response_array0 = data_grp[curr_slice][resp_type][data_type][:,cell_rois]



    #filter responses
    response_array1 = np.array([response_array0[idx,:] for idx,x in enumerate(trial_filter)\
                       if x])

    #re-arrange by configs
    resp_array_cfgs = np.empty((nrois,nconfigs,trials_to_keep))
    for cfg in range(nconfigs):
        tidx = np.where(trial_config[trial_filter] == cfg)
        for ridx in range(nrois):
            resp_array_cfgs[ridx,cfg,:] = response_array1[tidx,ridx]



    #---Perform statistical test for responsiveness
    tstat = np.zeros((nrois,nconfigs))
    pval = np.zeros((nrois,nconfigs))
    for ridx in range(nrois):
        for cidx in range(nconfigs):
            #get ttest stats by comparing df distribution to 0
            tstat[ridx,cidx], pval[ridx,cidx] = \
            stats.ttest_1samp(np.squeeze(resp_array_cfgs[ridx,cidx,:]),0)

    #put results in a common array
    if len(all_pval) == 0:
        all_pval = np.ravel(pval)
        all_tstat_sign = np.ravel(np.sign(tstat))
        sess_idx_array = np.ones(nrois)*sess_idx
        ntrials_array = trials_to_keep
    else:
        all_pval =np.hstack((all_pval,np.ravel(pval)))
        all_tstat_sign =np.hstack((all_tstat_sign,np.ravel(np.sign(tstat))))
        sess_idx_array = np.hstack((sess_idx_array,np.ones(nrois)*sess_idx))
        ntrials_array = np.hstack((ntrials_array,trials_to_keep))

20190909_JC110
**All configs matched to have 38 trials**
Opening:/n/coxfs01/2p-data/JC110/20190909/FOV1_zoom4p0x/scenes_combined/responses/traces102_s2p/files/trial_response_array.hdf5
ROIs:148
Cells:115
20190911_JC110
**All configs matched to have 38 trials**
Opening:/n/coxfs01/2p-data/JC110/20190911/FOV1_zoom4p0x/scenes_combined/responses/traces102_s2p/files/trial_response_array.hdf5
ROIs:228
Cells:176
20190913_JC110
**All configs matched to have 39 trials**
Opening:/n/coxfs01/2p-data/JC110/20190913/FOV1_zoom4p0x/scenes_combined/responses/traces102_s2p/files/trial_response_array.hdf5
ROIs:227
Cells:180
20190916_JC110
**All configs matched to have 37 trials**
Opening:/n/coxfs01/2p-data/JC110/20190916/FOV1_zoom4p0x/scenes_combined/responses/traces102_s2p/files/trial_response_array.hdf5
ROIs:239
Cells:155
20190918_JC110
**All configs matched to have 38 trials**
Opening:/n/coxfs01/2p-data/JC110/20190918/FOV1_zoom4p0x/scenes_combined/responses/traces102_s2p/files/trial_response_array.hdf5

In [4]:
#correct p-values for a set FDR 
[dummy,fdr_pval,alphacSidak,alphacBonf] = multitest.multipletests(all_pval,method ='fdr_bh')

#reshape array
fdr_pval = np.reshape(fdr_pval,(sess_idx_array.size,nconfigs))
contrast_sign = np.reshape(all_tstat_sign,(sess_idx_array.size,nconfigs))

In [5]:
#go through session again
# sess_idx = 0
# sess = all_session_list[sess_idx]
for sess_idx, sess in enumerate(all_session_list):

    print(sess)

    i1 = findOccurrences(sess,'_')[0]

    opts.animalid = sess[i1+1:]
    opts.session = sess[0:i1]

    #% Set up file paths:    
    acquisition_dir = os.path.join(opts.rootdir, opts.animalid, opts.session, opts.acquisition)
    responses_dir = os.path.join(acquisition_dir, opts.combined_run,'responses',traceid)
    src_file_dir = os.path.join(responses_dir, 'files')

    #*open file*
    fdr_fn = 'pval_fdr_corrected_%s_%s_arrays.hdf5'%(resp_type,data_type)
    fdr_filepath = os.path.join(src_file_dir, fdr_fn)
    fdr_grp = h5py.File(fdr_filepath, 'w')
    fdr_grp.attrs['num_trials'] = ntrials_array[sess_idx]
    
    #get corrected values for this session
    sess_rows = np.where(sess_idx_array == sess_idx)[0]
    fdr_pval_sess = fdr_pval[sess_rows,:]
    contrast_sign_sess = contrast_sign[sess_rows,:]
    
    #save to file
    p_dset = fdr_grp.create_dataset('fdr_pval',fdr_pval_sess.shape,\
                                      fdr_pval_sess.dtype)
    p_dset[...] = fdr_pval_sess
    
    s_dset = fdr_grp.create_dataset('contrast_sign',contrast_sign_sess.shape,\
                                      contrast_sign_sess.dtype)
    s_dset[...] = contrast_sign_sess

    fdr_grp.close()

20190909_JC110
20190911_JC110
20190913_JC110
20190916_JC110
20190918_JC110
20190920_JC110
20190925_JC110
20190930_JC110
20191003_JC110
20190927_JC111
20191002_JC111
20191004_JC111
20191016_JC113
20191019_JC113
20191021_JC113
20191023_JC113
20191025_JC113
20191029_JC113
20191101_JC113
20191106_JC113
20191024_JC117
20191028_JC117
20191030_JC117
20191107_JC117
20191118_JC117
20191115_JC120
20191116_JC120
20191124_JC120
20191125_JC120
20191126_JC120
20191127_JC120
20191119_JC121
20190619_JC080
20190624_JC085
20190704_JC085
20190712_JC085
20190621_JC091
20190625_JC091
20190701_JC091
20190703_JC091
20190705_JC091
20190621_JC097
20190628_JC097
20190702_JC097
20190704_JC097
20190708_JC097


In [6]:
#total # of positive tests
qval_thresh = 1e-3
print(np.sum(np.ravel(fdr_pval<qval_thresh)))
#number of positive tests that would be false positives
print(np.sum(np.ravel(fdr_pval<qval_thresh))*qval_thresh)

9017
9.017


In [41]:
fdr_pval.shape

(5950, 45)